# 部署应用

针对应用,python有一套完善的项目部署方案,从打包到环境隔离,再到监控,一应俱全.如果没有特殊需求,完全可以跳过容器直接使用.当然了如果运维希望使用docker这类容器部署以限制各个项目的资源使用量时,得益于`zipapp`和`pip`,python也同样简单易于部署.


## 打包与分发

python应用其实并不一定需要打包就可以分发部署,但即便如此python也有多样的打包分发方案,比较主流的有:

+ zipapp
+ Nuitka

我们以一个简单的flask例子作为要打包的内容,这个例子代码在<>中,其结构为:

```shell
packapp\
       |--app\
             |--__init__.py
             |--api\
                   |--__init__.py
                   |--core.py
                   |--index.py
```

+ `app/__init__.py`

    ```python
    from flask import Flask
    from .api import restapi


    def main() -> None:
        app = Flask("packapp")
        app.config["JSON_AS_ASCII"] = False
        restapi.init_app(app)
        app.run()

    ```

+ `app/api/__init__.py`

    ```python
    from .core import restapi
    from .index import *
    ```

+ `app/api/core.py`

    ```python
    from flask import Blueprint, Flask
    from flask.views import MethodView
    from typing import Callable, Type, Optional


    class APIView:
        def __init__(self, name: str = 'restapi', url_prefix: str = "/api", app: Optional[Flask] = None) -> None:
            self.restapi = Blueprint(name, __name__, url_prefix=url_prefix)
            if app:
                self.init_app(app)

        def register(self, url: str) -> Callable[[Type[MethodView]], Type[MethodView]]:
            def wrap(clz: Type[MethodView]) -> Type[MethodView]:
                self.restapi.add_url_rule(url, view_func=clz.as_view(clz.__name__))
                return clz
            return wrap

        def init_app(self, app: Flask) -> None:
            app.register_blueprint(self.restapi)


    restapi = APIView()
    __all__ = ["restapi", "APIView"]
    ```

+ `app/api/index.py`

    ```python
    from sanic.response import json
    from sanic.views import HTTPMethodView
    from .core import restapi

    @restapi.register("/")
    class IndexAPI(HTTPMethodView):

        async def get(self, request):
            result = {
                "description": "测试api"
            }
            return json(result,ensure_ascii=False)

    __all__ = ["IndexAPI"]
    ```


### 不打包

python实际上在2.5版本就已经支持直接执行内含`__main__.py`的文件夹了,实际上后面的zipapp和egg都是由这个特性来的.因此直接拿这个文件夹发布也是可以的.只要在目录下加上`__main__.py`即可:

```shell
packapp\
       |--__main__.py
       |--app\
             |--__init__.py
             |--api\
                   |--__init__.py
                   |--core.py
                   |--index.py
```

+ `__main__.py`

    ```python
    from app import main

    if __name__ == "__main__":
        main()
    ```

在`packapp`上一级目录中执行:

```bash
python packapp
```

### zipapp打包

python3.5提供了一种打包分发的方式--`zipapp`([PEP-441](https://peps.python.org/pep-0441/)).它可以将写好的项目打包成`.pyz`文件,这样就可以简单的将项目四处分发了.注意,这种方式最好是打包纯python代码,这样不容易因为平台不同而出现无法使用的情况.如果有c扩展,那么最好单独抽出来写成模块利用pip单独安装.

`.pyz`文件并不能独立运行,依然依赖python环境.因此如果不用`docker`打包的话最好使用虚拟环境让项目在虚拟环境中运行.


`zipapp`的用法如下:

在packapp上一级目录中执行

```bash
python -m zipapp packapp -m "app:main" 
```

`myapp`是一个项目文件夹,并非模块,我们使用`-m`指定使用其中的哪个模块的哪个方法作为入口

同时也可以使用`-p`指定一个字符串作为`Shebang`

```bash
python -m zipapp packapp -m "app:main" -p "/user/bin/env python3"
```

`zipapp`本质是一个用zip打包项目的工具,它的定位其实是简化版`jar`.一个打包好的二进制文件远比文件夹好分发使用.这也是go语言的核心竞争力之一.现在python有了这样一个工具,虽然使用起来还是要配合虚拟机和pip包管理工具,但已经很够用了.

执行:

```bash
python packapp.pyz
```

### Nuitka

上面2种方式都需要依赖环境,当运行环境中有依赖没有安装或者由依赖冲突就无法执行,这就需要使用后面提到的'虚拟环境'.另一个跨平台的打包工具[Nuitka](https://github.com/Nuitka/Nuitka)可以将环境一并打包到一起,但它并不能交叉编译所以需要每个平台打包一次然后分发,这也已经是很方便了.

针对上面这个项目我们在`packapp`文件夹下使用如下命令:

```shell
python -m nuitka --standalone --onefile --include-package-data="flask" --output-filename=packapp __main__.py
```
其中`--standalone`指定nuitka打包是包含python运行环境;`--onefile`指定打包的结果为一个可执行文件;`--include-package-data`指定包含除了`.py`文件以外其他资源文件的包依赖;`--output-filename`指定输出的可执行文件名;`__main__.py`指定程序的入口文件.

nuitka默认会递归的寻找依赖,并将依赖一并打包,编译出可执行文件.

执行:

```bash
./packapp
```

如果需要打包成特定平台的可执行文件形式,可以增加一些对应的参数,这些参数可以在`nuitka --help`中查看.

后面介绍用python写应用的部分我会在各种应用中穿插使用`Nuitka`打包的注意事项.


## 使用虚拟环境部署

项目部署运行时不可能通过常规手段激活虚拟环境.而事实上也不需要,其实要使用虚拟环境只要指定好用虚拟环境的python解释器运行项目了.比如有个虚拟环境建在`~/VENV`文件夹.那么就可以直接使用这个文件夹下的python解释器直接使用.

`~/VENV/bin/python myapp.pyz`

## `*`批量部署

python的运维神器[fabric](https://github.com/fabric/fabric),用它可以实现对远程服务器的批量部署操作

一些使用方法和心得可以看[我的博客](https://blog.hszofficial.site/introduce/2020/10/23/%E4%BD%BF%E7%94%A8Fabric%E5%81%9A%E8%BF%9C%E7%A8%8B%E6%89%B9%E9%87%8F%E9%83%A8%E7%BD%B2/)

## `*`服务监控

python的另一运维神器[supervisor](http://supervisord.org/),配合[cesi](https://github.com/gamegos/cesi)可以很好的监控管理项目进程.具体的可以看我的[这篇博文](https://blog.hszofficial.site/introduce/2015/02/19/%E4%BD%BF%E7%94%A8Supervisor%E5%81%9A%E6%9C%8D%E5%8A%A1%E7%9B%91%E6%8E%A7%E5%92%8C%E7%AE%A1%E7%90%86/)
